# Importing HTML Tables Into Pandas

## Setting up the program
Here we are importing the libraries needed to scrape data from a webpage, including urllib2, pandas, and BeautifulSoup.

After grabbing the URL and putting it in the object `html`, we pass it into `bsObj`, a BeautifulSoup object:

In [51]:
import urllib2
import pandas as pd
import html5lib
from pandas import DataFrame
from bs4 import BeautifulSoup
html = urllib2.urlopen('http://www.pythonscraping.com/pages/page3.html')
bsObj = BeautifulSoup(html)

## Operating on BeautifulSoup object:

Here, we use the `.find()` method to find the tag for a table, with the keywords for "id=giftlist".

The `.get_text()` is added to the end to strip out all of the html tags to show only the text (and to take up less space in the cell below!).

In [203]:
giftable = bsObj.find("table", {"id":"giftList"}).get_text()
giftable

u'\n\nItem Title\n\nDescription\n\nCost\n\nImage\n\n\nVegetable Basket\n\nThis vegetable basket is the perfect gift for your health conscious (or overweight) friends!\nNow with super-colorful bell peppers!\n\n$15.00\n\n\n\n\nRussian Nesting Dolls\n\nHand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! 8 entire dolls per set! Octuple the presents!\n\n$10,000.52\n\n\n\n\nFish Painting\n\nIf something seems fishy about this painting, it\'s because it\'s a fish! Also hand-painted by trained monkeys!\n\n$10,005.00\n\n\n\n\nDead Parrot\n\nThis is an ex-parrot! Or maybe he\'s only resting?\n\n$0.50\n\n\n\n\nMystery Box\n\nIf you love suprises, this mystery box is for you! Do not place on light-colored surfaces. May cause oil staining. Keep your friends guessing!\n\n$1.50\n\n\n\n'

### Selecting table rows

Instead of selecting the whole table, we can select an individual row.  Here, I have passed the second row ("id=gift2") to the object `gift2` by selecting "tr" for "table row".

We can see the content for the second row of the table (item, description, and cost) is returned.

In [163]:
gift2 = bsObj.find("tr", {"id":"gift2"}).get_text()
print gift2


Russian Nesting Dolls

Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! 8 entire dolls per set! Octuple the presents!

$10,000.52





The data type that is passed out is unicode, so operating on objects returned by BeautifulSoup takes a little practice.  Passing the information into a list, dictionary, or tuple is not as direct.

In [204]:
type(gift2)

unicode

In [141]:
r2c1 = bsObj.findAll('table')[0].findAll('tr')[2].findAll('td')[0].get_text()
r2c1 = str(r2c1).strip('n\'')
print r2c1
type(r2c1)


Russian Nesting Dolls



str

### Siblings of Table Data

The code below will print out all of the product rows (tags included).  Similar to the code above, we have selected the table tag with "id=giftList".  The `tr.next_siblings` will create an iterable object where the next table row (`tr`) will be returned.

In [206]:
for sibling in bsObj.find("table", {"id":"giftList"}).tr.next_siblings:
    print(sibling)



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

## Hacking together a DataFrame

Since I had difficulty operating on the BeautifulSoup objects, I decided to do it the way I could make it work:

### Selecting a single cell in an HTML table:

without calling any parents, children, or descendants, I figured out you can simply select a single cell by slicing the table, then calling `findAll()` on the table row, seleting a slice of that row, and the using `findAll()` again on the table data (`td`).  It's ugly, but doing this I could get each cell individually.

Below, you can see the data from the second row (not counting the header), and the second column:

In [207]:
print bsObj.findAll('table')[0].findAll('tr')[2].findAll('td')[1].get_text()


Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! 8 entire dolls per set! Octuple the presents!



Since all of the data is returned as unicode, I transformed it into a string, and then stripped the 'new line' out. This still left blank strings inbetween the words, so I removed them using a list comprehension. 

Since I did not need the "Image" column (the files come up as `NaN`), I removed that from the list.

In [208]:
header = str(bsObj.findAll('table')[0].findAll('tr')[0].get_text()).strip('\n')
header = header.splitlines()
header = [x for x in header if x != '']
header.remove('Image')
print header

['Item Title', 'Description', 'Cost']


Okay, this is super ugly. Here is each cell in the table (thankfully, it's a small table), passed into an object named after the location (r1c3 is "Row 1, Column 3").

In [209]:
r1c1 = str(bsObj.findAll('table')[0].findAll('tr')[1].findAll('td')[0].get_text()).strip('\n')
r1c2 = str(bsObj.findAll('table')[0].findAll('tr')[1].findAll('td')[1].get_text()).strip('\n')
r1c3 = str(bsObj.findAll('table')[0].findAll('tr')[1].findAll('td')[2].get_text()).strip('\n')
r2c1 = str(bsObj.findAll('table')[0].findAll('tr')[2].findAll('td')[0].get_text()).strip('\n')
r2c2 = str(bsObj.findAll('table')[0].findAll('tr')[2].findAll('td')[1].get_text()).strip('\n')
r2c3 = str(bsObj.findAll('table')[0].findAll('tr')[2].findAll('td')[2].get_text()).strip('\n')
r3c1 = str(bsObj.findAll('table')[0].findAll('tr')[3].findAll('td')[0].get_text()).strip('\n')
r3c2 = str(bsObj.findAll('table')[0].findAll('tr')[3].findAll('td')[1].get_text()).strip('\n')
r3c3 = str(bsObj.findAll('table')[0].findAll('tr')[3].findAll('td')[2].get_text()).strip('\n')
r4c1 = str(bsObj.findAll('table')[0].findAll('tr')[4].findAll('td')[0].get_text()).strip('\n')
r4c2 = str(bsObj.findAll('table')[0].findAll('tr')[4].findAll('td')[1].get_text()).strip('\n')
r4c3 = str(bsObj.findAll('table')[0].findAll('tr')[4].findAll('td')[2].get_text()).strip('\n')
r5c1 = str(bsObj.findAll('table')[0].findAll('tr')[5].findAll('td')[0].get_text()).strip('\n')
r5c2 = str(bsObj.findAll('table')[0].findAll('tr')[5].findAll('td')[1].get_text()).strip('\n')
r5c3 = str(bsObj.findAll('table')[0].findAll('tr')[5].findAll('td')[2].get_text()).strip('\n')
r5c3

'$1.50'

### Creating Columns

Now that I have all of my data as string data types, I can create my DataFrame as I know how to do with my limited experience.

In [194]:
col1 = [r1c1, r2c1, r3c1, r4c1, r5c1]
col2 = [r1c2, r2c2, r3c2, r4c2, r5c2]
col3 = [r1c3, r2c3, r3c3, r4c3, r5c3]

Since we have the columns, we can use the zip function to put them all together into an object named `html_table`.

In [195]:
html_table = zip(col1, col2, col3)
print html_table

[('Vegetable Basket', 'This vegetable basket is the perfect gift for your health conscious (or overweight) friends!\nNow with super-colorful bell peppers!', '$15.00'), ('Russian Nesting Dolls', 'Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! 8 entire dolls per set! Octuple the presents!', '$10,000.52'), ('Fish Painting', "If something seems fishy about this painting, it's because it's a fish! Also hand-painted by trained monkeys!", '$10,005.00'), ('Dead Parrot', "This is an ex-parrot! Or maybe he's only resting?", '$0.50'), ('Mystery Box', 'If you love suprises, this mystery box is for you! Do not place on light-colored surfaces. May cause oil staining. Keep your friends guessing!', '$1.50')]


### Creating a simple DataFrame object.

The `html_table` object will be passed into the pandas DataFrame function. Instead of zipping the header in with the rest of the columns, I just used the 'columns=' to name them.

In [199]:
df = DataFrame(data = html_table, columns=['Item Title', 'Description', 'Cost'])
df

,Item Title,Description,Cost
0,Vegetable Basket,This vegetable basket is the perfect gift for ...,$15.00
1,Russian Nesting Dolls,"Hand-painted by trained monkeys, these exquisi...","$10,000.52"
2,Fish Painting,"If something seems fishy about this painting, ...","$10,005.00"
3,Dead Parrot,This is an ex-parrot! Or maybe he's only resting?,$0.50
4,Mystery Box,"If you love suprises, this mystery box is for ...",$1.50


In [212]:
type(df)

pandas.core.frame.DataFrame

## Another way to make a dataframe

I was running into trouble using BeautifulSoup to get the data I needed, so in my searches for solutions, I found the `html5lib` in the Pandas documentation.  It is not included in your standard Anaconda package, but can easily be installed.

With this package, we can simply use the pd.read_html function and pass it the URL:

In [211]:
table_read = pd.read_html('http://www.pythonscraping.com/pages/page3.html', header=0, index_col=None)

In [17]:
table_read[0]

,Item Title,Description,Cost,Image
0,Vegetable Basket,This vegetable basket is the perfect gift for ...,$15.00,NaN
1,Russian Nesting Dolls,"Hand-painted by trained monkeys, these exquisi...","$10,000.52",NaN
2,Fish Painting,"If something seems fishy about this painting, ...","$10,005.00",NaN
3,Dead Parrot,This is an ex-parrot! Or maybe he's only resting?,$0.50,NaN
4,Mystery Box,"If you love suprises, this mystery box is for ...",$1.50,NaN


In [169]:
df2 = DataFrame(table_read[0])

In [170]:
type(df2)

pandas.core.frame.DataFrame

Voila! A dataframe object with not much effort. I figured this is not the way we needed to complete the assignment, but thought it was a neat tool to have in the Python/web scraping toolbox.

In [171]:
print df2

              Item Title                                        Description  \
0       Vegetable Basket  This vegetable basket is the perfect gift for ...   
1  Russian Nesting Dolls  Hand-painted by trained monkeys, these exquisi...   
2          Fish Painting  If something seems fishy about this painting, ...   
3            Dead Parrot  This is an ex-parrot! Or maybe he's only resting?   
4            Mystery Box  If you love suprises, this mystery box is for ...   

         Cost  Image  
0      $15.00    NaN  
1  $10,000.52    NaN  
2  $10,005.00    NaN  
3       $0.50    NaN  
4       $1.50    NaN  


In [210]:
print df2.index
print df2.columns

Int64Index([0, 1, 2, 3, 4], dtype='int64')
Index([u'Item Title', u'Description', u'Cost', u'Image'], dtype='object')
